# `MEDUSA`
aka. Dynamic-Prospective LCA aka. Union(premise, temporalis)

In [1]:
from bw_temporalis import easy_timedelta_distribution
from edge_extractor import EdgeExtracter
from medusa_tools import *
import bw2data as bd
import bw2calc as bc
import numpy as np
import pandas as pd

In [2]:
bd.projects.set_current("medusa_hydrogen_example")

### Setup of our Example

In [3]:
bd.Database('temporalis-bio').write({
    ('temporalis-bio', "CO2"): {
        "type": "emission",
        "name": "carbon dioxide",
        "temporalis code": "co2",
    },
    ('temporalis-bio', "CH4"): {
        "type": "emission",
        "name": "methane",
        "temporalis code": "ch4",
    },
})

bd.Database('background_2023').write({
    ('background_2023', 'electricity_mix'): {
        'name': 'Electricity mix',
        'exchanges': [
            {
                'amount': 1,
                'type': 'production',
                'input': ('background_2023', 'electricity_mix'),
            },
            {
                'amount': 1,
                'type': 'technosphere',
                'input': ('background_2023', 'electricity_wind'),
            },
        ]
    },
    ('background_2023', 'electricity_wind'): {
        'name': 'Electricity production, wind',
        'exchanges': [
            {
                'amount': 1,
                'type': 'production',
                'input': ('background_2023', 'electricity_wind'),
            },
            {
                'amount': 1,
                'type': 'biosphere',
                'input': ('temporalis-bio', 'CO2'),
            },
        ]
    }
})

bd.Database('background_2020').write({
    ('background_2020', 'electricity_mix'): {
        'name': 'Electricity mix',
        'exchanges': [
            {
                'amount': 1,
                'type': 'production',
                'input': ('background_2020', 'electricity_mix'),
            },
            {
                'amount': 1,
                'type': 'technosphere',
                'input': ('background_2020', 'electricity_wind'),
            },
        ]
    },
    ('background_2020', 'electricity_wind'): {
        'name': 'Electricity production, wind',
        'exchanges': [
            {
                'amount': 1,
                'type': 'production',
                'input': ('background_2020', 'electricity_wind'),
            },
            {
                'amount': 468745,
                'type': 'biosphere',
                'input': ('temporalis-bio', 'CO2'),
            },
        ]
    }
})

bd.Database('foreground').write({
    ('foreground', 'heat_from_hydrogen'): {
        'name': 'Heat production, hydrogen',
        'exchanges': [
            {
                'amount': 1,
                'type': 'production',
                'input': ('foreground', 'heat_from_hydrogen'),
            },
            {
                'amount': 1,
                'type': 'technosphere',
                'input': ('foreground', 'electrolysis'),
                'temporal_distribution': # e.g. because some hydrogen was stored in the meantime
                    easy_timedelta_distribution(
                    start=-2,
                    end=0, # Range includes both start and end
                    resolution="Y",  # M for months, Y for years, etc.
                    steps=2,
                ),
            },
        ]
    },
    ('foreground', 'electrolysis'): {
        'name': 'Hydrogen production, electrolysis',
        'exchanges': [
            {
                'amount': 1,
                'type': 'production',
                'input': ('foreground', 'electrolysis'),
            },
            {
                'amount': 1,
                'type': 'technosphere',
                'input': ('background_2023', 'electricity_mix'),
            },
        ]
    },
})

100%|██████████| 2/2 [00:00<?, ?it/s]

Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<?, ?it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<?, ?it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<?, ?it/s]


Vacuuming database 


In [4]:
bd.Method(("GWP", "example")).write([
    (('temporalis-bio', "CO2"), 1),
    (('temporalis-bio', "CH4"), 25),
])

In [5]:
demand = {('foreground', 'heat_from_hydrogen'): 1}
gwp = ('GWP', 'example')

# Static LCA

In [6]:
slca = bc.LCA(demand, gwp)
slca.lci()
slca.lcia()
print(f'Static LCA score: {slca.score}')

Static LCA score: 1.0


# `MEDUSA` LCA

A MEDUSA LCA builds upon a static LCA, but adds a temporal dimensions, linking to prospective LCA databases. Similarly to a `Temporalis LCA`, the supply chain graph is traversed, taking into account temporal distributions of the edges. 

For now, only the foreground system is assumed to have temporal distributions. Therefore, we define a filter function, that tells to EdgeExtracter (which is doing the actual graph traversal and saves the edges with respective timestamps), when a database that is known to have no temporal distributions (i.e., the prospective background databases) is reached, so that the traversal can be stopped.

In [7]:
SKIPPABLE = [node.id for node in bd.Database('background_2020')] + [
    node.id for node in bd.Database('background_2023')
]

def filter_function(database_id: int) -> bool:
    return database_id in SKIPPABLE

Now we can do the graph traversal and create a timeline of edges:

In [8]:
eelca = EdgeExtracter(slca, edge_filter_function=filter_function)
timeline = eelca.build_edge_timeline()

Starting graph traversal
Calculation count: 3


Next, we define a dictionary containing the dates of our prospective background databases. Using this, we can create a timeline dataframe. 

The dates of the edges are mapped to the prospective background databases; interpolation is used for dates in between the dates of the background databases. The default is linear interpolation, another currently included option is "nearest", choosing the next best fitting database.

In [9]:
database_date_dict = {
            datetime.strptime("2020", "%Y"): 'background_2020',
            datetime.strptime("2023", "%Y"): 'background_2023',
        }

timeline_df = create_grouped_edge_dataframe(timeline, database_date_dict.keys(), interpolation_type="linear")
timeline_df

,date,year,producer,producer_name,consumer,consumer_name,amount,interpolation_weights
0,2021-01-01,2021,27,Electricity mix,32,"Hydrogen production, electrolysis",0.5,"{2020: 0.666058394160584, 2023: 0.333941605839..."
1,2021-01-01,2021,32,"Hydrogen production, electrolysis",31,"Heat production, hydrogen",0.5,"{2020: 0.666058394160584, 2023: 0.333941605839..."
2,2023-01-01,2023,27,Electricity mix,32,"Hydrogen production, electrolysis",0.5,{2023: 1}
3,2023-01-01,2023,31,"Heat production, hydrogen",-1,-1,1.0,{2023: 1}
4,2023-01-01,2023,32,"Hydrogen production, electrolysis",31,"Heat production, hydrogen",0.5,{2023: 1}


Now, we want to create a datapackage that takes care of relinking processes to our prospective databases. To do so, we need to provide the timeline dataframe, the dict of prospective databases and corresponding years, and a new dictionary that defines at which point in time our functional unit is assessed *(We can probably include this information in the database_date_dict in the future, but for now, this works)*.

In [10]:
demand_timing_dict = create_demand_timing_dict(timeline_df, demand)

dp = create_datapackage_from_edge_timeline(timeline_df, database_date_dict, demand_timing_dict)

Finally, we just have to reformat our input data for the LCA, add our datapackage containing the patches, and run the lca.

In [11]:
fu, data_objs, remapping = prepare_medusa_lca_inputs(demand=demand, demand_timing_dict=demand_timing_dict, method=gwp) 
lca = bc.LCA(fu, data_objs = data_objs + [dp], remapping_dicts=remapping)
lca.lci()
lca.lcia()

Let's take a look at the results:

In [12]:
print('New MEDUSA LCA Score:', lca.score)
print('Old static LCA Score:', slca.score)

New MEDUSA LCA Score: 78053.21897810219
Old static LCA Score: 1.0
